In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

# Create dataset

In [53]:
shakspeare_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

In [54]:
filepath = keras.utils.get_file("shakespeares.txt", shakspeare_url)

1122304/1115394 [==============================] - 0s 0us/step


In [55]:
with open(filepath) as f:
    shakespeare_text = f.read()

In [56]:
len(shakespeare_text)

1115394

In [57]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [58]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [59]:
tokenizer.fit_on_texts(shakespeare_text)

In [60]:
max_id = len(tokenizer.word_index)

In [61]:
dataset_size = tokenizer.document_count

In [62]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [63]:
train_size = dataset_size * 90 // 100

In [64]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [65]:
n_steps = 25
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [66]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [67]:
batch_size = 10
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [68]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [69]:
dataset = dataset.prefetch(1)

In [70]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(10, 25, 39) (10, 25)


# Create model

In [71]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

In [72]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, None, 128)         64896     
_________________________________________________________________
gru_5 (GRU)                  (None, None, 128)         99072     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 39)          5031      
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)

Epoch 1/10
  3053/100385 [..............................] - ETA: 53:16 - loss: 2.0826